# Credit Card Fraud Detection Project
*Author: Rekula Prudhvi Raj*

This notebook leverages the [Kaggle Credit Card Fraud Detection dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download), which contains **284,807** anonymized transactions with only **492** labeled as fraud (≈0.17%). We’ll walk through:

1. **Loading & Exploration** – quick data overview and class distribution  
2. **Preprocessing** – scaling, missing-value checks, and feature engineering  
3. **Class Imbalance Handling** – using SMOTE to balance the minority class  
4. **Model Training** – comparing Logistic Regression, Decision Tree, Random Forest, XGBoost, and HistGradientBoosting  
5. **Evaluation** – precision, recall, F1-score, and ROC-AUC metrics with a focus on minimizing false positives  

In [2]:
# Optional: install/upgrade xgboost
!pip install --upgrade xgboost --quiet

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
print('Imports loaded')

import warnings
warnings.filterwarnings('ignore')

Imports loaded


In [4]:
df = pd.read_csv('creditcard.csv')
print('Data shape:', df.shape)
print(df['Class'].value_counts(normalize=True))

Data shape: (284807, 31)
Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64


In [5]:
# Preprocess: check missing, scale Amount, drop Time
assert df.isnull().sum().sum() == 0, 'Missing values!'
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])
df.drop(columns=['Time'], inplace=True)
print('Preprocessing done')

Preprocessing done


In [6]:
X = df.drop(columns=['Class'])
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print('Train:', X_train.shape, 'Test:', X_test.shape)

Train: (227845, 29) Test: (56962, 29)


In [7]:
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print('Resampled train:', X_train_res.shape)

Resampled train: (454902, 29)


In [8]:
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=500, n_jobs=-1, random_state=42),
    'Decision Tree': DecisionTreeClassifier(class_weight='balanced', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, scale_pos_weight=(len(y_train)-sum(y_train))/sum(y_train), n_jobs=-1, use_label_encoder=False, eval_metric='auc', random_state=42),
    'HistGradientBoosting': HistGradientBoostingClassifier(max_iter=100, early_stopping=True, random_state=42)
}

In [9]:
results = {}
for name, model in models.items():
    print(f"\n=== Training {name} ===")
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test)
    proba = model.predict_proba(X_test)[:, 1]
    report = classification_report(y_test, preds, output_dict=True)
    auc = roc_auc_score(y_test, proba)
    results[name] = {
        'precision_fraud': report['1']['precision'],
        'recall_fraud': report['1']['recall'],
        'f1_fraud': report['1']['f1-score'],
        'roc_auc': auc
    }
    print(f"{name} ROC AUC: {auc:.4f}")


=== Training Logistic Regression ===
Logistic Regression ROC AUC: 0.9700

=== Training Decision Tree ===
Decision Tree ROC AUC: 0.8917

=== Training Random Forest ===
Random Forest ROC AUC: 0.9753

=== Training XGBoost ===
XGBoost ROC AUC: 0.9784

=== Training HistGradientBoosting ===
HistGradientBoosting ROC AUC: 0.9683


In [12]:
import pandas as pd
summary_df = pd.DataFrame(results).T
print('\n=== Model Comparison ===')
print(summary_df.sort_values('roc_auc', ascending=False))


=== Model Comparison ===
                      precision_fraud  recall_fraud  f1_fraud   roc_auc
XGBoost                      0.480226      0.867347  0.618182  0.978391
Random Forest                0.852632      0.826531  0.839378  0.975314
Logistic Regression          0.056285      0.918367  0.106070  0.970028
HistGradientBoosting         0.470270      0.887755  0.614841  0.968320
Decision Tree                0.359813      0.785714  0.493590  0.891653


## Interpretation of Model Comparison & Recommendations

**XGBoost** (AUC: 0.978391, Precision: 0.480226, Recall: 0.867347, F1: 0.618182)  
- Highest AUC: best overall discrimination; use when you need top ranking and can accept moderate false positives.

**Random Forest** (AUC: 0.975314, Precision: 0.852632, Recall: 0.826531, F1: 0.839378)  
- Best balance (highest F1): recommended for all-round performance when you need both few false positives and few missed frauds.

**Logistic Regression** (AUC: 0.970028, Precision: 0.056285, Recall: 0.918367, F1: 0.106070)  
- Very high recall: use when catching every fraud is critical and you can tolerate many false alarms.

**HistGradientBoosting** (AUC: 0.968320, Precision: 0.470270, Recall: 0.887755, F1: 0.614841)  
- Fast training with high recall: alternative when you need early stopping and quicker iterations.

**Decision Tree** (AUC: 0.891653, Precision: 0.359813, Recall: 0.785714, F1: 0.493590)  
- Fastest to train but lowest performance: useful for quick prototyping or when interpretability is most important.

**Choosing a model:**  
- If you need fewest false alarms (high precision): **Random Forest**.  
- If you must catch all fraud (high recall): **Logistic Regression** or **HistGradientBoosting** (then raise threshold to improve precision).  
- If you want best overall ranking (AUC): **XGBoost** (consider threshold tuning for precision).  
- For rapid prototyping: **Decision Tree**, then upgrade for production.  

Consider stacking your top models for even better results!
